# CNN Model for Stock market Ratio Prediction

## Packages 

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from numpy import array 
##Removes warning for chained assignments
pd.options.mode.chained_assignment = None  # default='warn'

2022-09-01 20:14:07.798535: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-01 20:14:07.798606: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Import Data


In [2]:
df = pd.read_csv('../raw_data/ratios.csv')
df

,Date,ALB_ZBRA,NRG_CPRT,DE_BXP,NRG_STE,NRG_XRAY,SO_NLOK,PEG_O
0,2020-08-31,0.311845,0.310544,2.548054,0.203834,0.727693,2.136432,0.882281
1,2020-09-01,0.332974,0.306303,2.687319,0.203522,0.722940,2.094566,0.855546
2,2020-09-02,0.332368,0.315056,2.629334,0.204743,0.734758,2.125291,0.872526
3,2020-09-03,0.339772,0.310060,2.534974,0.201707,0.728923,2.210971,0.848504
4,2020-09-04,0.356920,0.300754,2.536138,0.199887,0.707646,2.152835,0.854765
...,...,...,...,...,...,...,...,...
492,2022-08-15,0.813028,0.321488,4.111073,0.197467,1.123670,3.277218,0.920803
493,2022-08-16,0.814154,0.323634,4.118669,0.200499,1.158238,3.283272,0.933379
494,2022-08-17,0.815450,0.327442,4.154118,0.201342,1.191888,3.347435,0.935173
495,2022-08-18,0.816374,0.333768,4.222490,0.204492,1.196920,3.371537,0.950678


126 days of stock prices ratios for 231 different stock pairs, will build model to look at first ratio. 

## Create Array of single ratios and create last 20 days columns. 

In [3]:
df.set_index('Date',inplace=True)

In [4]:
frames = []
for columns in df:
    frames.append(pd.DataFrame(df[columns]))

for frame in frames:
    for i in range(1,21):
        frame[f't-{i}'] = frame[frame.columns[0]].shift(i)
    frame.dropna(inplace=True)


    
frames[0]

,ALB_ZBRA,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,...,t-11,t-12,t-13,t-14,t-15,t-16,t-17,t-18,t-19,t-20
Date,,,,,,,,,,,,,,,,,,,,,
2020-09-29,0.330664,0.320942,0.323581,0.327588,0.317605,0.372109,0.376984,0.376953,0.386490,0.377806,...,0.383597,0.372984,0.368015,0.367225,0.366863,0.356920,0.339772,0.332368,0.332974,0.311845
2020-09-30,0.348574,0.330664,0.320942,0.323581,0.327588,0.317605,0.372109,0.376984,0.376953,0.386490,...,0.375199,0.383597,0.372984,0.368015,0.367225,0.366863,0.356920,0.339772,0.332368,0.332974
2020-10-01,0.330732,0.348574,0.330664,0.320942,0.323581,0.327588,0.317605,0.372109,0.376984,0.376953,...,0.377806,0.375199,0.383597,0.372984,0.368015,0.367225,0.366863,0.356920,0.339772,0.332368
2020-10-02,0.340060,0.330732,0.348574,0.330664,0.320942,0.323581,0.327588,0.317605,0.372109,0.376984,...,0.386490,0.377806,0.375199,0.383597,0.372984,0.368015,0.367225,0.366863,0.356920,0.339772
2020-10-05,0.348504,0.340060,0.330732,0.348574,0.330664,0.320942,0.323581,0.327588,0.317605,0.372109,...,0.376953,0.386490,0.377806,0.375199,0.383597,0.372984,0.368015,0.367225,0.366863,0.356920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-15,0.813028,0.812049,0.790774,0.788457,0.764204,0.741221,0.717486,0.710539,0.733862,0.754330,...,0.683022,0.685980,0.692162,0.688442,0.699419,0.677911,0.679211,0.674544,0.678144,0.689156
2022-08-16,0.814154,0.813028,0.812049,0.790774,0.788457,0.764204,0.741221,0.717486,0.710539,0.733862,...,0.669475,0.683022,0.685980,0.692162,0.688442,0.699419,0.677911,0.679211,0.674544,0.678144
2022-08-17,0.815450,0.814154,0.813028,0.812049,0.790774,0.788457,0.764204,0.741221,0.717486,0.710539,...,0.754330,0.669475,0.683022,0.685980,0.692162,0.688442,0.699419,0.677911,0.679211,0.674544


The CNN model can be used to model the ratios but we first have to pre-process the data into a format the model will understand this consists of a series of smaller samples for this I will start with using the Monday to Friday 

In [6]:
Big_X = []
Big_Y = []
for frame in frames:
    X = []
    for i in range(len(frame[frame.columns[0]])):
        arr = []
        for j in range(1,frame.shape[1]):
            arr.append(frame[f't-{j}'][i])
        X.append(arr)
    Y = frame[frame.columns[0]].tolist()
    Big_X.append(X)
    Big_Y.append(Y)


In [9]:
model = Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(20,1)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer='adam', loss='mae')

2022-09-01 20:14:53.732065: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-01 20:14:53.732130: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-01 20:14:53.732151: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-QMR9Q7U): /proc/driver/nvidia/version does not exist
2022-09-01 20:14:53.732475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
Big_X = array(Big_X)
Big_X[0].shape

(477, 20)

In [35]:
for x in Big_X:
    x = x.reshape((x.shape[0], x.shape[1], 1))
Big_X[0].shape

(477, 20)

In [36]:
X.shape,Y.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
X_train = X[0:457]
X_test = X[457:]
Y_train = Y[0:457]
Y_test = Y[457:]

In [ ]:
X_train.shape,Y_train.shape

In [ ]:
model.fit(X_train,Y_train, epochs=100, verbose=0)

In [ ]:
X_test[1].shape

In [ ]:
preds = []
for i in range(len(X_test)):    
    pred = model.predict(X_test[i].reshape(1,20,1))
    preds.append(pred[0][0])
preds

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mape = []
for i in range(len(preds)):
    mape.append(mean_absolute_percentage_error(Y_test[i],preds[i]))
mape

In [ ]:
sum(mape)/len(mape)

In [ ]:
fig, ax = plt.subplots(1,1,figsize = (13,10))
ax.plot(A['Ratio'[0:100]])
ax.plot(A.index[-20:],Y_test)
ax.plot(A.index[-20:],preds)

In [ ]:
preds